In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import json 

### Загрузка необходимых данных 

In [2]:
with open("data/test_object_lu.json", "r") as file:
    plan = json.load(file)
    
partition = cv2.imread('data/partition.jpg')
v_corners = np.load("data/v_corners.npy")
h_corners = np.load("data/h_corners.npy")
panorama = cv2.imread("data/image_1024_aligned_rgb.png")
depth = np.load("data/depth_map.npy") * 1000 # in millimeters 

In [3]:
print(depth.shape)
print(depth.max())
print(depth)
# Нормализация данных с камеры глубины
scaled_depth = ((depth - np.min(depth)) / (np.max(depth) - np.min(depth)) * 255).astype(np.uint8)
cv2.imshow('Depth Image', scaled_depth)
cv2.waitKey(0)
cv2.destroyAllWindows()

(512, 1024)
6101.31
[[1214.8892 1214.8892 1214.8892 ... 1214.8892 1214.8892 1214.8892]
 [1214.9348 1214.9348 1214.9348 ... 1214.9348 1214.9348 1214.9348]
 [1215.0264 1215.0264 1215.0264 ... 1215.0264 1215.0264 1215.0264]
 ...
 [1600.1882 1600.1882 1600.1882 ... 1600.1882 1600.1882 1600.1882]
 [1600.0677 1600.0677 1600.0677 ... 1600.0677 1600.0677 1600.0677]
 [1600.0076 1600.0076 1600.0076 ... 1600.0076 1600.0076 1600.0076]]


In [4]:
# Загрузить изображения
alpha = 0.7

# Преобразовать глубину в трехканальное изображение
depth_colored = cv2.applyColorMap((scaled_depth * 255).astype(np.uint8), cv2.COLORMAP_JET)

# Наложить изображения
overlay = cv2.addWeighted(panorama, alpha, depth_colored, 1 - alpha, 0)

# Вывести результат
cv2.imshow('Overlay', overlay)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
cv2.imshow("source_image", panorama)
cv2.waitKey(0)
cv2.destroyAllWindows()
print(v_corners)

[[  30.914364  222.41164 ]
 [  30.914364  298.64883 ]
 [ 363.15613   140.81924 ]
 [ 363.15613   392.54047 ]
 [ 672.78357   149.66058 ]
 [ 672.78357   383.4991  ]
 [ 801.5895    137.09825 ]
 [ 801.5895    396.26495 ]
 [ 845.40314    76.09795 ]
 [ 845.40314   451.50623 ]
 [1008.1279    221.95033 ]
 [1008.1279    299.23853 ]]


In [6]:
ceil_height = 250
area_mappings = {
    '0': (v_corners[0:4], 2050),
    '1': (v_corners[2:6], 1640),
    '2': (v_corners[4:8], 400),
    '3': (v_corners[6:10], 4370),
    '4': (v_corners[8:12], 1650),
    '5': (np.concatenate((v_corners[10:], v_corners[:2])), 6020),
}
print(area_mappings)

{'0': (array([[ 30.914364, 222.41164 ],
       [ 30.914364, 298.64883 ],
       [363.15613 , 140.81924 ],
       [363.15613 , 392.54047 ]], dtype=float32), 2050), '1': (array([[363.15613, 140.81924],
       [363.15613, 392.54047],
       [672.78357, 149.66058],
       [672.78357, 383.4991 ]], dtype=float32), 1640), '2': (array([[672.78357, 149.66058],
       [672.78357, 383.4991 ],
       [801.5895 , 137.09825],
       [801.5895 , 396.26495]], dtype=float32), 400), '3': (array([[801.5895 , 137.09825],
       [801.5895 , 396.26495],
       [845.40314,  76.09795],
       [845.40314, 451.50623]], dtype=float32), 4370), '4': (array([[ 845.40314,   76.09795],
       [ 845.40314,  451.50623],
       [1008.1279 ,  221.95033],
       [1008.1279 ,  299.23853]], dtype=float32), 1650), '5': (array([[1008.1279  ,  221.95033 ],
       [1008.1279  ,  299.23853 ],
       [  30.914364,  222.41164 ],
       [  30.914364,  298.64883 ]], dtype=float32), 6020)}


In [7]:
start = int(v_corners[0][0])
print("start of the line:", start)
end = int(v_corners[2][0])
print("end of the line:", end)
line = []
for x in range(start, end+1, 32):
    line.append([x, h_corners[:, x][0]])
print(line)

start of the line: 30
end of the line: 363
[[30, 222.71733], [62, 192.55711], [94, 169.15553], [126, 150.01512], [158, 136.72841], [190, 129.05185], [222, 124.01126], [254, 122.649155], [286, 123.07381], [318, 126.99274], [350, 135.25735]]


In [8]:
for i in range(0, len(v_corners) - 2, 2):
    print(v_corners[i], v_corners[i + 1], v_corners[i + 2], v_corners[i + 3])

[ 30.914364 222.41164 ] [ 30.914364 298.64883 ] [363.15613 140.81924] [363.15613 392.54047]
[363.15613 140.81924] [363.15613 392.54047] [672.78357 149.66058] [672.78357 383.4991 ]
[672.78357 149.66058] [672.78357 383.4991 ] [801.5895  137.09825] [801.5895  396.26495]
[801.5895  137.09825] [801.5895  396.26495] [845.40314  76.09795] [845.40314 451.50623]
[845.40314  76.09795] [845.40314 451.50623] [1008.1279   221.95033] [1008.1279   299.23853]


In [9]:
print(h_corners)

[[221.67708 221.43863 221.38431 ... 222.38773 223.78665 223.0257 ]
 [299.86584 298.79175 299.72705 ... 298.76703 297.46896 299.3693 ]]


In [10]:
print(panorama.shape)

(512, 1024, 3)


In [11]:
import numpy as np
from e2p import e2p
import utils

def e2p_coor(eq_coor, fov_deg, u_deg, v_deg, eq_hw, out_hw):
    in_x, in_y = eq_coor
    u = np.linspace(-np.pi, np.pi, num=w, dtype=np.float32) # Широта 
    v = np.linspace(np.pi, -np.pi, num=h, dtype=np.float32) / 2 # Долгота 
    grid = np.stack(np.meshgrid(u, v), axis=-1)
    
    u, v = grid[int(in_y), int(in_x), :]
    x_max, y_max = np.tan(np.radians(fov_deg[0]) / 2), np.tan(np.radians(fov_deg[1]) / 2)
    
    pt = (abs(np.tan(u) + x_max), 
          abs(np.tan(v) - y_max))

    grid_size = (x_max * 2 / out_hw[1], 
                 y_max * 2 / out_hw[0])
    
    return pt[0] / grid_size[0], pt[1] / grid_size[1]

points = [
    [363.15613, 140.81924],
    [363.15613, 392.54047],
    [672.78357, 149.66058],
    [672.78357, 383.4991],
]
# area, fov_deg, u_deg, v_deg, in_rot
views = [['0', (120, 120), 0, 0, 0],
         # ['1', (120, 150), 80, 0, 0],
         # ['2', (120, 150), 80, 0, 0],
         # ['3', (130, 150), 150, 0, 0],
         # ['4', (130, 150), 150, 0, 0],
         # ['5', (150, 140), -110, 0, 0],
         ]

         # ['floor', (130, 140), 0, 110, 0],
         # ['ceil', (110, 110), 0, -110, 0]
equirectangular_img = panorama
h, w = equirectangular_img.shape[:2]

for area, fov_deg, u_deg, v_deg, in_rot in views:
    out_hw = (
        (h * fov_deg[1]) // 180 * 2,
        (w * fov_deg[0]) // 360 * 2,
    )
    # Вызов функции e2p
    pers_img = e2p(equirectangular_img, fov_deg, u_deg, v_deg, out_hw, in_rot_deg=0, mode='bilinear')
    for pt in points:
        point = (e2p_coor(pt, fov_deg, u_deg, v_deg, (h, w), out_hw))
        print(point)
        cv2.circle(pers_img, (int(point[0]), int(point[1])), 5, (0, 0, 255), -1)
    # Сохранение изображений
    cv2.imwrite(f"views/{area}_proj.jpg", pers_img) 

(86.6566080464152, 171.7496131583997)
(86.6566080464152, 560.2852384625255)
(638.2329423347452, 189.85125184436075)
(638.2329423347452, 537.2721725577104)


In [12]:
import numpy as np
from e2p import e2p


equirectangular_img = panorama
h, w = equirectangular_img.shape[:2]
print(h, w)
# face_k, fov_deg, u_deg, v_deg, in_rot
views = [['f', (120, 120), 0, 0, 0],
         ['r', (120, 150), 80, 0, 0],
         ['b', (130, 150), 150, 0, 0],
         ['l', (150, 140), -110, 0, 0],
         ['u', (130, 140), 0, 110, 0],
         ['d', (110, 110), 0, -110, 0]]
# Углы обзора
# fov_deg = (120, 120)
# u_deg = 0
# v_deg = 90

# Размер выходного изображения
# out_hw = (1200, 900)
for face_k, fov_deg, u_deg, v_deg, in_rot in views:
    out_hw = (
        (512 * fov_deg[1]) // 180 * 2,
        (1024 * fov_deg[0]) // 360 * 2,
    )
    # Угол поворота входного изображения
    in_rot_deg = 0
    
    # Режим интерполяции
    mode = 'bilinear'
    
    # Вызов функции e2p
    pers_img = e2p(equirectangular_img, fov_deg, u_deg, v_deg, out_hw, in_rot_deg, mode)
    bgr_scaled_depth = cv2.cvtColor(scaled_depth, cv2.COLOR_GRAY2BGR)
    pers_depth = e2p(bgr_scaled_depth, fov_deg, u_deg, v_deg, out_hw, in_rot_deg, mode)
    
    # Визуализация результата (при необходимости) 
    # cv2.imshow('Perspective Image', pers_img)
    # cv2.imshow('Perspective Depth', pers_depth)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    # Сохранение изображений
    cv2.imwrite(f"frblud/{face_k}_depth.jpg", pers_depth)
    cv2.imwrite(f"frblud/{face_k}_image.jpg", pers_img) 

512 1024


In [16]:
import open3d as o3d
import numpy as np
import cv2

# Получение ширины изображения
# Взятие половины изображения по горизонтали
rgb_image = cv2.imread('frblud/u_image.jpg')
depth_image = cv2.imread('frblud/u_depth.jpg')
width = rgb_image.shape[1]
# Создание облака точек из данных камеры RGB-D
rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
    o3d.geometry.Image(rgb_image), 
    o3d.geometry.Image(depth_image)
)

pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
    rgbd_image,
    o3d.camera.PinholeCameraIntrinsic(o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault))
# Flip it, otherwise the pointcloud will be upside down
pcd.transform([[1, 0, 0, 0], 
               [0, -1, 0, 0], 
               [0, 0, -1, 0], 
               [0, 0, 0, 1]])
# Визуализация облака точек
o3d.visualization.draw_geometries([pcd])

In [14]:
# outliers removal
cl, ind = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=20.0)
pcd = pcd.select_by_index(ind)

# estimate normals
pcd.estimate_normals()
pcd.orient_normals_to_align_with_direction()
 
# surface reconstruction
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=10, n_threads=1)[0]

# rotate the mesh
rotation = mesh.get_rotation_matrix_from_xyz((np.pi, 0, 0))
mesh.rotate(rotation, center=(0, 0, 0))

# save the mesh
o3d.io.write_triangle_mesh(f'./mesh.obj', mesh)
o3d.visualization.draw_geometries([pcd])

In [21]:
from defisheye import Defisheye
import cv2 

dtype = 'linear'
format = 'fullframe'
fov = 160
pfov = 120

img = "frblud/f_image.jpg"

obj = Defisheye(img, dtype=dtype, format=format, fov=fov, pfov=pfov)

# To use the converted image in memory

new_image = obj.convert()

cv2.imshow("1", new_image)
cv2.waitKey(0)
cv2.destroyAllWindows()